In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification
from data_configs.configs import *
import statistics
import numpy as np

config = house_votes_84_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
knn_model = KNN(config)
null_model = NullModelClassification(config=config)

### Data Load and Preprocessing ###

In [2]:
raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

In [3]:
data_3

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,2
695,2,1,1,1,2,1.0,1,1,1,2
696,5,10,10,3,7,3.0,8,10,2,4
697,4,8,6,4,3,4.0,10,6,1,4


## KNN Model ##

### Hyperparameter Tuning ###

In [4]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [5]:
data_train

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
82,5,2,1,1,2,1.0,3,1,1,2
51,5,3,3,4,2,4.0,3,4,1,4
220,1,1,1,2,2,1.0,3,1,1,2
559,5,1,1,1,2,1.0,2,1,1,2
544,2,1,3,2,2,1.0,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...
71,6,10,2,8,10,2.0,7,8,10,4
106,10,10,10,8,2,10.0,4,1,1,4
270,8,4,7,1,3,10.0,3,9,2,4
435,10,8,10,1,3,10.0,5,1,1,4


#### Tuning k ####

In [6]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
        
        predictions_1 = knn_model.knn_classifier(data_val, train_set_1, k=k)['Predicted Class']
        score_1 = Evaluation().zero_one_loss(data_val[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average 0-1 Loss score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average 0-1 loss score of {scores_dict[best_k]}")


Average 0-1 Loss score with k=1: 0.05428571428571429
Average 0-1 Loss score with k=2: 0.08
Average 0-1 Loss score with k=3: 0.03357142857142857
Average 0-1 Loss score with k=4: 0.045714285714285714
Average 0-1 Loss score with k=5: 0.03142857142857143
Average 0-1 Loss score with k=6: 0.03571428571428571
Average 0-1 Loss score with k=7: 0.02785714285714285
Average 0-1 Loss score with k=8: 0.03
Average 0-1 Loss score with k=9: 0.027142857142857142
Average 0-1 Loss score with k=10: 0.03
Average 0-1 Loss score with k=11: 0.02857142857142857
Average 0-1 Loss score with k=12: 0.03142857142857143
Average 0-1 Loss score with k=13: 0.029285714285714283
Average 0-1 Loss score with k=14: 0.03357142857142857
Best k is 9 with the lowest average 0-1 loss score of 0.027142857142857142


### Model Performance ###

In [7]:
zero_one_loss_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
    
    # Train and evaluate 
    predictions_1 = knn_model.knn_classifier(test_set, train_set, k=best_k)['Predicted Class']
    
    zero_one_loss_score = Evaluation.zero_one_loss(test_set[config['target_column']], predictions_1)
    precision_score = Evaluation.precision(test_set[config['target_column']], predictions_1)
    recall_score = Evaluation.recall(test_set[config['target_column']], predictions_1)
    f1_score = Evaluation.f1_score(test_set[config['target_column']], predictions_1)
    
    zero_one_loss_scores.append(zero_one_loss_score)
    precision_scores.append(precision_score)
    recall_scores.append(recall_score)
    f1_scores.append(f1_score)

    null_model_prediction = null_model.naive_classifier(test_set)
    null_model_score = Evaluation.zero_one_loss(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 


average_01_score = sum(zero_one_loss_scores) / len(zero_one_loss_scores)
average_precision_score = sum(precision_scores) / len(precision_scores)
average_recall_score = sum(recall_scores) / len(recall_scores)
average_f1_score = sum(f1_scores) / len(f1_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)

print(f"Average null model 0-1 loss score: {average_null_model_score}")
print(f"Average KNN 0-1 score for k={best_k}: {average_01_score}")
print(f"Average Precision score for k={best_k}: {average_precision_score}")
print(f"Average Recall score for k={best_k}: {average_recall_score}")
print(f"Average F1 score for k={best_k}: {average_f1_score}")


Average null model 0-1 loss score: 0.35062724014336916
Average KNN 0-1 score for k=9: 0.03577956989247312
Average Precision score for k=9: 0.9645357411379856
Average Recall score for k=9: 0.9642204301075268
Average F1 score for k=9: 0.9641369555377652


## Edited KNN ##

### Hyperparameter Tuning ###

#### Tuning k ####

In [8]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
        
        edited_train_set = knn_model.edited_knn_classificaton(train_set_1)

        predictions_1 = knn_model.knn_classifier(data_val, edited_train_set, k=k)['Predicted Class']
        score_1 = Evaluation().zero_one_loss(data_val[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average 0-1 loss score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average 0-1 loss score of {scores_dict[best_k]}")


Average 0-1 loss score with k=1: 0.034999999999999996
Average 0-1 loss score with k=2: 0.042142857142857135
Average 0-1 loss score with k=3: 0.03214285714285715
Average 0-1 loss score with k=4: 0.037142857142857144
Average 0-1 loss score with k=5: 0.032857142857142856
Average 0-1 loss score with k=6: 0.03428571428571429
Average 0-1 loss score with k=7: 0.03142857142857143
Average 0-1 loss score with k=8: 0.03142857142857143
Average 0-1 loss score with k=9: 0.030000000000000006
Average 0-1 loss score with k=10: 0.032857142857142856
Average 0-1 loss score with k=11: 0.03142857142857143
Average 0-1 loss score with k=12: 0.035714285714285705
Average 0-1 loss score with k=13: 0.032857142857142856
Average 0-1 loss score with k=14: 0.03571428571428571
Best k is 9 with the lowest average 0-1 loss score of 0.030000000000000006


In [9]:
zero_one_loss_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
    
    edited_train_set = knn_model.edited_knn_classificaton(train_set)

    predictions_1 = knn_model.knn_classifier(test_set, edited_train_set, k=best_k)['Predicted Class']

    zero_one_loss_score = Evaluation.zero_one_loss(test_set[config['target_column']], predictions_1)
    precision_score = Evaluation.precision(test_set[config['target_column']], predictions_1)
    recall_score = Evaluation.recall(test_set[config['target_column']], predictions_1)
    f1_score = Evaluation.f1_score(test_set[config['target_column']], predictions_1)
    
    zero_one_loss_scores.append(zero_one_loss_score)
    precision_scores.append(precision_score)
    recall_scores.append(recall_score)
    f1_scores.append(f1_score)

    null_model_prediction = null_model.naive_classifier(test_set)
    null_model_score = Evaluation.zero_one_loss(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 

average_01_score = sum(zero_one_loss_scores) / len(zero_one_loss_scores)
average_precision_score = sum(precision_scores) / len(precision_scores)
average_recall_score = sum(recall_scores) / len(recall_scores)
average_f1_score = sum(f1_scores) / len(f1_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)

print(f"Average null model 0-1 score: {average_null_model_score}")
print(f"Average KNN 0-1 score for k={best_k}: {average_01_score}")
print(f"Average Precision score for k={best_k}: {average_precision_score}")
print(f"Average Recall score for k={best_k}: {average_recall_score}")
print(f"Average F1 score for k={best_k}: {average_f1_score}")

Average null model 0-1 score: 0.35062724014336916
Average KNN 0-1 score for k=9: 0.040788530465949824
Average Precision score for k=9: 0.9594643546065822
Average Recall score for k=9: 0.9592114695340502
Average F1 score for k=9: 0.9590275641565421


## Condensed Knn ##

#### Tuning k ####

In [10]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
        
        condesed_train_set = knn_model.condensed_knn_classification(train_set_1)
        predictions_1 = knn_model.knn_classifier(data_val, condesed_train_set, k=k)['Predicted Class']
        score_1 = Evaluation().zero_one_loss(data_val[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average 0-1 Loss score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average 0-1 loss score of {scores_dict[best_k]}")


Average 0-1 Loss score with k=1: 0.07714285714285714
Average 0-1 Loss score with k=2: 0.155
Average 0-1 Loss score with k=3: 0.05714285714285714
Average 0-1 Loss score with k=4: 0.08714285714285713
Average 0-1 Loss score with k=5: 0.05071428571428571
Average 0-1 Loss score with k=6: 0.05142857142857142
Average 0-1 Loss score with k=7: 0.042142857142857135
Average 0-1 Loss score with k=8: 0.05
Average 0-1 Loss score with k=9: 0.07928571428571428
Average 0-1 Loss score with k=10: 0.04357142857142857
Average 0-1 Loss score with k=11: 0.14785714285714285
Average 0-1 Loss score with k=12: 0.08357142857142856
Average 0-1 Loss score with k=13: 0.22142857142857145
Average 0-1 Loss score with k=14: 0.22214285714285714
Best k is 7 with the lowest average 0-1 loss score of 0.042142857142857135


### Model Performance ###

In [11]:
zero_one_loss_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
    
    condesed_train_set = knn_model.condensed_knn_classification(train_set)

    predictions_1 = knn_model.knn_classifier(test_set, condesed_train_set, k=best_k)['Predicted Class']

    zero_one_loss_score = Evaluation.zero_one_loss(test_set[config['target_column']], predictions_1)
    precision_score = Evaluation.precision(test_set[config['target_column']], predictions_1)
    recall_score = Evaluation.recall(test_set[config['target_column']], predictions_1)
    f1_score = Evaluation.f1_score(test_set[config['target_column']], predictions_1)
    
    zero_one_loss_scores.append(zero_one_loss_score)
    precision_scores.append(precision_score)
    recall_scores.append(recall_score)
    f1_scores.append(f1_score)

    null_model_prediction = null_model.naive_classifier(test_set)
    null_model_score = Evaluation.zero_one_loss(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 

average_01_score = sum(zero_one_loss_scores) / len(zero_one_loss_scores)
average_precision_score = sum(precision_scores) / len(precision_scores)
average_recall_score = sum(recall_scores) / len(recall_scores)
average_f1_score = sum(f1_scores) / len(f1_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)

print(f"Average null model 0-1 score: {average_null_model_score}")
print(f"Average KNN 0-1 score for k={best_k}: {average_01_score}")
print(f"Average Precision score for k={best_k}: {average_precision_score}")
print(f"Average Recall score for k={best_k}: {average_recall_score}")
print(f"Average F1 score for k={best_k}: {average_f1_score}")

Average null model 0-1 score: 0.35062724014336916
Average KNN 0-1 score for k=7: 0.07698412698412699
Average Precision score for k=7: 0.9405529232164694
Average Recall score for k=7: 0.9230158730158731
Average F1 score for k=7: 0.9221589014291244
